In [1]:
# Import necessary packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [2]:
def grayscale(img):
    # Convert mpimg color space into 1-channel grayscale
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [3]:
def gaussian_blur(img, kernel_size=5):
    # Apply Gaussian blur with default kernel size
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [4]:
def canny(img, low_threshold, high_threshold):
    # Canny edge detection
    return cv2.Canny(img, low_threshold, high_threshold)

In [5]:
def polygon_roi(img):
    r, c, _ = image.shape
    vertices = np.array([[
        (20, r),                       # lower left
        (c // 2 - 20, r // 2 + 50),    # upper left
        (c // 2 + 20, r // 2 + 50),    # upper right
        (c - 20, r),                   # lower right
    ]], dtype=np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked = cv2.bitwise_and(img, mask)
    return masked

In [6]:
def hough_lines(img):
    # Distance resolution of the Hough grid (in pixels)
    rho = 2
    # Angular resolution of the Hough grid (in radians)
    theta = np.pi / 180
    # Min. # of votes required for a line
    threshold = 5
    # Min. # of pixels required for a line
    min_line_len = 20
    # Maximum gap between connectable line segments
    max_line_gap = 40
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

In [7]:
def detect_lines(img):
    # Obtain grayscale image
    gray = grayscale(img)
    # Apply Gaussian blur filter
    blurred = gaussian_blur(gray, 3)
    # Detect Canny edges
    edges = canny(blurred, 50, 200)
    # Mask region of interest
    roi = polygon_roi(edges)
    # Detect Hough lines
    lines = hough_lines(roi)
    return lines

In [8]:
def select_lines(lines, min_angle, max_angle, min_x, max_x):
    # Calculate tangent for each angle
    min_gradient = np.tan(np.radians(min_angle))
    max_gradient = np.tan(np.radians(max_angle))
    selected = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            # First we check the position of the line
            if min(x1, x2) < min_x or max(x1, x2) > max_x:
                continue
            # Vertical lines are not likely to be lane lines
            if x1 == x2:
                continue
            # We subtract y2 from y1 since y-axis is upside down
            m = (y1 - y2) / (x2 - x1)
            # For those selected lines, keep the following:
            # (1) gradient, (2) y-intercept, (3) segnemtn length,
            if min_gradient <= m <= max_gradient:
                selected.append(line)
    return selected

In [9]:
def aggregate_lines(lines):
    gradients, intercepts, weights = [], [], []
    min_y, max_y = 0, 560
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            # Keep min & max y values
            min_y = min(y1, y2) if min_y == 0 or min(y1, y2) < min_y else min_y
            max_y = max(y1, y2) if max(y1, y2) > max_y else max_y
            m = (y2 - y1) / (x2 - x1)
            b = y1 - m * x1
            # Use each line's length as its weight in averaging
            l = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            gradients.append(m)
            intercepts.append(b)
            weights.append(l)
    # Now we compute weighted (based on line length) average of the
    # gradient and y-intercept
    g, i, w = np.array(gradients), np.array(intercepts), np.array(weights)
    avg_m = np.average(g, weights=w)
    avg_b = np.average(i, weights=w)
    # Calculate low & high endpoints
    x1 = int((max_y - avg_b) / avg_m)
    x2 = int((min_y - avg_b) / avg_m)
    return x1, max_y, x2, min_y

In [10]:
def draw_lines(img, lines, c=(255, 0, 0), t=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), c, t)
    return img

In [11]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [12]:
def process_image(img):
    lines = detect_lines(img)
    # print("# of found lines: ", len(lines))
    left_candidates = select_lines(lines, 30, 45, 0, 480)
    # print("# of left lane line candidates: ", len(left_candidates))
    right_candidates = select_lines(lines, -45, -30, 480, 960)
    # print("# of right lane line candidates: ", len(right_candidates))

    line_img = np.zeros_like(img)
    # line_img = draw_lines(line_img, left_candidates, c=(0, 255, 0))
    # line_img = draw_lines(line_img, right_candidates, c=(255, 255, 0))
    if len(left_candidates) > 0:
        left_lane_line = aggregate_lines(left_candidates)
        line_img = draw_lines(line_img, [(left_lane_line, )], c=(255, 0, 0), t=10)
    if len(right_candidates) > 0:
        right_lane_line = aggregate_lines(right_candidates)
        line_img = draw_lines(line_img, [(right_lane_line, )], c=(0, 0, 255), t=10)
    annotated = weighted_img(line_img, img)
    return annotated

In [13]:
import os

filenames = os.listdir("test_images")
for filename in filenames:
    pathname = os.path.join(os.getcwd(), "test_images", filename)
    # Read image file
    image = mpimg.imread(pathname)
    lane_line_detection = process_image(image)
    cv2_image = cv2.cvtColor(lane_line_detection, cv2.COLOR_RGB2BGR)
    output_pathname = os.path.join(os.getcwd(), "test_images_output", filename)
    cv2.imwrite(output_pathname, cv2_image)

In [16]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

filenames = os.listdir("test_videos")
for filename in filenames:
    pathname = os.path.join(os.getcwd(), "test_videos", filename)
    output_pathname = os.path.join(os.getcwd(), "test_videos_output", filename)
    clip = VideoFileClip(pathname)
    output_clip = clip.fl_image(process_image)
    %time output_clip.write_videofile(output_pathname, audio=False)

[MoviePy] >>>> Building video /src/test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video /src/test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:05<00:00, 36.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.22 s, sys: 284 ms, total: 2.51 s
Wall time: 6.82 s
[MoviePy] >>>> Building video /src/test_videos_output/challenge.mp4
[MoviePy] Writing video /src/test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:15<00:00, 15.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/challenge.mp4 

CPU times: user 6.18 s, sys: 610 ms, total: 6.79 s
Wall time: 17.5 s
[MoviePy] >>>> Building video /src/test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video /src/test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:22<00:00, 29.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/solidYellowLeft.mp4 

CPU times: user 8.03 s, sys: 916 ms, total: 8.95 s
Wall time: 23.7 s
